<a href="https://colab.research.google.com/github/Walkneeer/NanoGPT/blob/main/rag_app_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 72.7 MB/s eta 0:00:00


# Create and run local RAG pipeline from scratch

In [2]:
import os
import requests

# get pdf doc
pdf_path = '/content/vessel-sanitation-program-2011-construction-guidelines.pdf'

# Downlad
if not os.path.exists(pdf_path):
  print(f'[INFO] file do not exist, download')
  url = 'https://www.mycruiseship.info/media/pdf/usph/vessel-sanitation-program-2011-construction-guidelines.pdf'
  filename = pdf_path
  response = requests.get(url)
  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
    print('[INFO] file is has been download')
  else:
    print('[INFO] fail to download')
else:
  print('[INFO] file exist')

[INFO] file do not exist, download
[INFO] file is has been download


In [3]:
import fitz
from tqdm.auto import tqdm
import re

def text_formater(text: str) -> str:
    # Replace all newlines and tabs with a space
    cleaned_text = re.sub(r'[\n\t]+', ' ', text)
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    # Strip any leading or trailing spaces
    cleaned_text = cleaned_text.strip()
    return cleaned_text

def open_pdf(pdf_path: str) -> list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_text = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    cleaned_text = text_formater(text)
    pages_and_text.append({'page_number': page_number,
                          'page_char_count': len(cleaned_text),
                          'page_token_count': len(cleaned_text) / 4,
                          'page_word_count': len(cleaned_text.split('. ')),
                          'text': cleaned_text})
  return pages_and_text


pages_and_text = open_pdf(pdf_path=pdf_path)
pages_and_text[19]

0it [00:00, ?it/s]

{'page_number': 19,
 'page_char_count': 2583,
 'page_token_count': 645.75,
 'page_word_count': 24,
 'text': 'VSP Construction Guidelines; 2 These guidelines are not meant to limit the introduction of new designs, materials, or technology for shipbuilding. A shipbuilder, owner, manufacturer, or other interested party may ask VSP to periodically review or revise these guidelines in relation to new information or technology. VSP reviews such requests in accordance with the criteria described in section 2.0. New cruise vessels must comply with all international code requirements (e.g., International Maritime Organization Conventions). Those include requirements of the following: • Safety of Life-at-Sea Convention. • International Convention for the Prevention of Pollution from Ships. • Tonnage and Load Line Convention. • International Electrical Code. • International Plumbing Code. • International Standards Organization. This document does not cross-reference related and sometimes overlapp

In [4]:
import pandas as pd

df = pd.DataFrame(pages_and_text)
df

,page_number,page_char_count,page_token_count,page_word_count,text
0,0,211,52.75,3,Vessel Sanitation Program 2011 Construction Gu...
1,1,685,171.25,4,Vessel Sanitation Program Centers for Disease ...
2,2,1290,322.50,11,Acknowledgments; i Acknowledgments VSP would l...
3,3,5532,1383.00,35,Contents; i Contents 1.0 Background and Purpos...
4,4,5815,1453.75,42,Contents; ii 6.3.2.1 Surfaces ...................
...,...,...,...,...,...
113,113,1627,406.75,14,VSP Construction Guidelines; 96 Fax: 770-488-4...
114,114,439,109.75,3,VSP Construction Guidelines; 97 Fax: 954-356-6...
115,115,61,15.25,1,VSP Construction Guidelines; 98 40.0 Vessel Pr...
116,116,967,241.75,1,VSP Construction Guidelines; 99 Vessel Name: D...


In [5]:
df.describe().round(2)

,page_number,page_char_count,page_token_count,page_word_count
count,118.00,118.00,118.00,118.00
mean,58.50,2209.03,552.26,15.97
std,34.21,1419.82,354.95,9.46
min,0.00,31.00,7.75,1.00
25%,29.25,1664.75,416.19,11.00
50%,58.50,1875.50,468.88,14.00
75%,87.75,2182.00,545.50,19.00
max,117.00,5936.00,1484.00,51.00


In [6]:
!pip install spacy

In [7]:
from spacy.lang.en import English
import random

nlp = English()
nlp.add_pipe("sentencizer")
doc = nlp('I like elp')
assert len(list(doc.sents)) == 1
list(doc.sents)

[I like elp]

In [8]:
for item in tqdm(pages_and_text):
  item['sentences'] = list(nlp(item['text']).sents)
  item['sentences'] = [str(sentence) for sentence in item['sentences']]
  item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/118 [00:00<?, ?it/s]

In [9]:
random.sample(pages_and_text, k=1)

[{'page_number': 1,
  'page_char_count': 685,
  'page_token_count': 171.25,
  'page_word_count': 4,
  'text': 'Vessel Sanitation Program Centers for Disease Control and Prevention 4770 Buford Highway, NE/MS F-59 Atlanta, GA 30341-3717 Phone: 770-488-7070 Fax: 770-488-4127 E-mail: vsp@cdc.gov Vessel Sanitation Program Centers for Disease Control and Prevention 1850 Eller Drive, Suite 101 Ft. Lauderdale, FL 33316-4201 Phone: 800-323-2132 or 954-356-6650 Fax: 954-356-6671 E-mail: vsp@cdc.gov The VSP 2011 Construction Guidelines and updates are available at http://www.cdc.gov/nceh/vsp. Use of trade names is for identification only and does not imply endorsement by the Centers for Disease Control and Prevention, the Public Health Service, or the U.S. Department of Health and Human Services.',
  'sentences': ['Vessel Sanitation Program Centers for Disease Control and Prevention 4770 Buford Highway, NE/MS F-59 Atlanta, GA 30341-3717 Phone: 770-488-7070 Fax: 770-488-4127 E-mail: vsp@cdc.gov Ve

### chunking

In [10]:
# Define split size to turn groups of sentences into chunks
split_size = 8

# Create a function to split ls of txt recursively into chunk size
def split_ls(input_list: list[str], slice_size: int) -> list[list[str]]:
  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_ls = list(range(25))
split_ls(test_ls, split_size)

[[0, 1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22, 23],
 [24]]

In [11]:
for item in tqdm(pages_and_text):
  item['sentences_chunks'] = split_ls(item['sentences'], split_size)
  item['num_chunks'] = len(item['sentences_chunks'])

  0%|          | 0/118 [00:00<?, ?it/s]

In [12]:
random.sample(pages_and_text, k=1)

[{'page_number': 103,
  'page_char_count': 2164,
  'page_token_count': 541.0,
  'page_word_count': 16,
  'text': 'VSP Construction Guidelines; 86 31.4 Recirculation and Filtration System 31.4.1 Compensation or Makeup Tank Install a compensation or make-up tank with an automatic level control system capable of holding an amount of water sufficient to ensure continuous operation of the filtration and DISINFECTION systems. This capacity must be equal to at least 3 times the total operating volume of the system. 31.4.2 Accessible Drain Install an ACCESSIBLE drain at the bottom of the tank to allow for complete draining of the tank. Install an access port for cleaning the tank and for the addition of batch halogenation and PH control chemicals. 31.4.3 Secondary Disinfection and pH Systems Design the system so that 100% of the water for the BABY-ONLY WATER FACILITY feature passes through filtration, halogenation, secondary DISINFECTION, and PH systems before returning to the BABY-ONLY WATER 

In [13]:
# split chunks into own item
import re

pages_and_chunks = []
for item in tqdm(pages_and_text):
  for sentences_chunks in item['sentences_chunks']:
    chunk_dict = {}
    chunk_dict['page_number'] = item['page_number']

    # join the sentences together into paragraph
    join_sentences = ' '.join(sentences_chunks).replace('  ', ' ').strip()

    chunk_dict['sentence_chunk'] = join_sentences

    chunk_dict['sentence_chunk_char_count'] = len(join_sentences)
    chunk_dict['sentence_word_count'] = len([word for word in join_sentences.split(' ')])
    chunk_dict['sentence_chunk_token_count'] = len(join_sentences) / 4

    pages_and_chunks.append(chunk_dict)
len(pages_and_chunks)

  0%|          | 0/118 [00:00<?, ?it/s]

237

In [14]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 10,
  'sentence_chunk': 'Contents; viii 21.2.1 Construction ............................................................................................................ 57 21.2.2 Pressure-washing System........................................................................................ 57 21.2.3 Handwashing Station .............................................................................................. 57 21.2.4 Ventilation............................................................................................................... 57 21.3 Garbage Holding Facilities ..........................................................................................57 21.3.1 Size and Location .................................................................................................... 57 21.3.2 Ventilation............................................................................................................... 57 21.3.3 Refrigerated Storage ................

# Embeding

In [15]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.5 MB/s eta 0:00:00


In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device='cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

embeddings = model.encode(sentences)
embeding_dict = dict(zip(sentences, embeddings))

for sentence, embeding in embeding_dict.items():
  print(f'{sentence}\n{embeding}\n')

This framework generates embeddings for each input sentence
[ 6.41692942e-03  7.04136817e-03 -2.81441920e-02  5.12471721e-02
 -8.93960800e-03  2.12669224e-02  2.30778418e-02 -1.44859785e-02
 -5.55310957e-03 -2.49297973e-02  4.53492738e-02  2.48959735e-02
 -3.07579394e-02  5.66225052e-02  6.32021800e-02 -5.62528111e-02
  5.16509749e-02  5.78279700e-03 -2.62116604e-02  1.31878909e-03
  1.99272353e-02 -1.30592426e-03 -2.28708494e-03  4.72541489e-02
 -3.72495241e-02 -2.85245627e-02 -4.10240702e-02 -1.57975890e-02
  3.17328097e-03 -8.74196528e-04 -2.96459366e-02  3.21501456e-02
  3.51344422e-02  1.09738167e-02  9.16707108e-07 -1.18588237e-03
 -2.53640655e-02 -7.92881101e-03 -5.09479223e-03  7.40658911e-03
  2.80067995e-02  1.06996251e-02  1.07513322e-02  2.76827794e-02
 -5.19132763e-02 -4.98179197e-02  5.34074046e-02  5.79067916e-02
  7.86073804e-02  7.73014799e-02 -1.01112127e-02 -6.35445789e-02
 -1.71579309e-02 -6.77370466e-03 -2.45818519e-03  2.61346102e-02
 -5.38513670e-03 -3.02865058e-

In [18]:
embeddings[1].shape

(768,)

In [19]:
pages_and_chunks[0]

{'page_number': 0,
 'sentence_chunk': 'Vessel Sanitation Program 2011 Construction Guidelines U.S. Department of Health and Human Services U.S. Public Health Service Centers for Disease Control and Prevention/ National Center for Environmental Health',
 'sentence_chunk_char_count': 211,
 'sentence_word_count': 28,
 'sentence_chunk_token_count': 52.75}

In [20]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks]
text_chunks[12]

CPU times: user 165 µs, sys: 27 µs, total: 192 µs
Wall time: 486 µs


'Contents; x 22.9.4 Paint or Stripe ......................................................................................................... 66 22.9.5 Steam Generation for Food Areas ........................................................................... 66 22.9.5.1 Nonpotable Water ............................................................................................ 66 22.10 Disinfection of the Potable Water System ...................................................................66 22.10.1 Before Placed in Service ..................................................................................... 66 22.10.2 Free Chlorine Solution ........................................................................................ 66 22.10.3 Documentation .................................................................................................... 66 22.11 Potable Water Pressure Tanks .....................................................................................67 22.1

In [21]:
%%time

model.to('cpu')

for item in tqdm(pages_and_chunks):
  item['embedding'] = model.encode(item['sentence_chunk'])

  0%|          | 0/237 [00:00<?, ?it/s]

CPU times: user 4min 15s, sys: 1.88 s, total: 4min 16s
Wall time: 4min 22s


In [22]:
#%%time

#text_chunks_embeddings = model.encode(text_chunks, batch_size=16, convert_to_tensor=True)

In [23]:
#text_chunks_embeddings

In [24]:
text_chunks_embeddings_df = pd.DataFrame(pages_and_chunks)
save_path_df = 'embedings_ls.csv'
text_chunks_embeddings_df.to_csv(save_path_df, index=False)

In [25]:
text_chunks_embeddings_df = pd.read_csv(save_path_df)
text_chunks_embeddings_df.head()

,page_number,sentence_chunk,sentence_chunk_char_count,sentence_word_count,sentence_chunk_token_count,embedding
0,0,Vessel Sanitation Program 2011 Construction Gu...,211,28,52.75,[-1.32696787e-02 3.04913819e-02 -2.41265781e-...
1,1,Vessel Sanitation Program Centers for Disease ...,685,94,171.25,[-1.91081520e-02 2.11132392e-02 -2.58935634e-...
2,2,Acknowledgments; i Acknowledgments VSP would l...,1291,212,322.75,[ 2.81663612e-02 1.94496345e-02 -5.18616959e-...
3,3,Contents; i Contents 1.0 Background and Purpos...,5532,237,1383.00,[-6.05050521e-03 2.47094352e-02 -1.68723706e-...
4,4,Contents; ii 6.3.2.1 Surfaces ...................,5815,220,1453.75,[-2.83990037e-02 -8.32910165e-02 1.76016539e-...


# RAG

In [26]:
import torch
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load CSV and process embeddings
text_chunks_embeddings_df = pd.read_csv('embedings_ls.csv')

# Convert the 'embedding' column (currently string) into a list of floats
text_chunks_embeddings_df['embedding'] = text_chunks_embeddings_df['embedding'].apply(lambda x: [float(i) for i in x.strip('[]').split()])

# Convert the list of embeddings into a tensor with a specified data type (e.g., float32)
embeddings = torch.tensor(text_chunks_embeddings_df['embedding'].tolist(), dtype=torch.float32, device=device)

# If you need to keep the rest of the DataFrame in a dictionary format:
pages_and_chunks = text_chunks_embeddings_df.to_dict(orient='records')

# View the processed embeddings
text_chunks_embeddings_df['embedding']

,embedding
0,"[-0.0132696787, 0.0304913819, -0.0241265781, -..."
1,"[-0.019108152, 0.0211132392, -0.0258935634, -0..."
2,"[0.0281663612, 0.0194496345, -0.0518616959, -0..."
3,"[-0.00605050521, 0.0247094352, -0.0168723706, ..."
4,"[-0.0283990037, -0.0832910165, 0.0176016539, -..."
...,...
232,"[0.0295634158, 0.0783538595, -0.0278103184, 0...."
233,"[-0.0231864247, 0.0195034482, -0.0306529272, -..."
234,"[-0.0600455999, -0.0779071525, -0.0339961834, ..."
235,"[-0.022683315, -0.0288872328, -0.038009312, -0..."


In [27]:
embeddings.shape

torch.Size([237, 768])

In [28]:
# Create model
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device=device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## semantic search pipeline

In [29]:
from sentence_transformers import util
import torch

# Define query
query = 'handwash'

# Embed query
query_embedding = embed_model.encode(query, convert_to_tensor=True)

from time import perf_counter as timer

# Start the timer
start_time = timer()

# Compute dot scores (dot product between query and other embeddings)
dot_scores = util.dot_score(query_embedding, embeddings)

# End the timer
end_time = timer()

# Print the result
print(f'[INFO] time: {end_time - start_time}')
print(dot_scores)

# Extract top 5 results
top_result_dot_product = torch.topk(dot_scores, k=5)
print(top_result_dot_product)

[INFO] time: 0.00040169600003991945
tensor([[ 0.4128,  0.4058,  0.1832,  0.0417,  0.2887,  0.4390,  0.0753,  0.2571,
          0.1902,  0.3831,  0.4165,  0.1482,  0.3684,  0.1579,  0.1862,  0.2587,
          0.0582,  0.1331,  0.3571,  0.1928,  0.2904,  0.1095,  0.1916,  0.2839,
         -0.0033,  0.0640,  0.0364,  0.0461,  0.1254,  0.1578,  0.0836,  0.1067,
          0.1917,  0.0631,  0.1472,  0.1686,  0.0935,  0.1956,  0.0217,  0.0888,
          0.2132,  0.0984,  0.1984,  0.2785,  0.0829,  0.2778,  0.2881,  0.1456,
          0.1759,  0.2757,  0.2385,  0.1630,  0.2298,  0.2093,  0.2689,  0.1048,
          0.1526,  0.2010,  0.1266,  0.1801,  0.1856,  0.1610,  0.2683,  0.0827,
          0.1854,  0.2960,  0.2086,  0.4251,  0.2551,  0.2010,  0.2824,  0.2497,
          0.0746,  0.0542,  0.1387,  0.1687,  0.1552,  0.1878,  0.4919,  0.3254,
          0.4640,  0.5294,  0.3693,  0.1553,  0.3503,  0.3559,  0.3522,  0.2197,
          0.2909,  0.1377,  0.0859,  0.1120,  0.1586,  0.1965,  0.1324,  

In [30]:
top_indice = top_result_dot_product[1]
top_indice

tensor([[122, 141, 210,  81, 214]])

In [31]:
top_text_chunks = text_chunks_embeddings_df.iloc[122]
top_text_chunks['sentence_chunk']

'Beverage stations are excluded. 17.5.4 Equipment and Supplies The handwashing station must include a handwash sink, soap dispenser, and single-use paper towel dispenser. Electric hand dryers can be installed in addition to paper towel dispensers. Waste receptacles must be provided in close proximity to the handwash sink and sized to accommodate the quantity of paper towel waste generated. The handwashing station may be decorative but must be nonabsorbent, durable, and EASILY CLEANABLE. 17.5.5 Automatic Handwashing System An automatic handwashing system in lieu of a handwash sink is acceptable. 17.5.6 Sign Each handwashing station must have a sign advising passengers to wash hands before eating. A pictogram can be used in lieu of words on the sign.'

In [32]:
vector1 = torch.tensor([1,2,3], dtype=torch.float32)
vector2 = torch.tensor([2,3,4], dtype=torch.float32)
vector3 = torch.tensor([-7,20,9], dtype=torch.float32)
vector4 = torch.tensor([1,2,3], dtype=torch.float32)

# Calculate the dot product
dot_product1 = torch.dot(vector1, vector2)
dot_product2 = torch.dot(vector1, vector3)
dot_product3 = torch.dot(vector1, vector4)

# Print the result
print(f"Dot product between vector1 and vector2: {dot_product1}")
print(f"Dot product between vector1 and vector3: {dot_product2}")
print(f"Dot product between vector1 and vector4: {dot_product3}")

Dot product between vector1 and vector2: 20.0
Dot product between vector1 and vector3: 60.0
Dot product between vector1 and vector4: 14.0


In [33]:
import torch

# Define the vectors
vector1 = torch.tensor([1, 2, 3], dtype=torch.float32)
vector2 = torch.tensor([2, 5, 4], dtype=torch.float32)
vector3 = torch.tensor([-7, 20, 9], dtype=torch.float32)
vector4 = torch.tensor([1, 2, 3], dtype=torch.float32)

# Calculate the cosine similarity
cosine_similarity1 = torch.nn.functional.cosine_similarity(vector1, vector2, dim=0)
cosine_similarity2 = torch.nn.functional.cosine_similarity(vector1, vector3, dim=0)
cosine_similarity3 = torch.nn.functional.cosine_similarity(vector1, vector4, dim=0)

# Print the results
print(f"Cosine similarity between vector1 and vector2: {cosine_similarity1}")
print(f"Cosine similarity between vector1 and vector3: {cosine_similarity2}")
print(f"Cosine similarity between vector1 and vector4: {cosine_similarity3}")

Cosine similarity between vector1 and vector2: 0.9561828374862671
Cosine similarity between vector1 and vector3: 0.6965451836585999
Cosine similarity between vector1 and vector4: 0.9999998807907104


In [43]:
from timeit import default_timer as timer
import torch
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict

def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer = None,
                                n_resources: int = 5,
                                print_time: bool = True):
    """
    Embeds a query with the embed model and retrieves relevant resources.
    """
    # Embed query
    query_embedding = embed_model.encode(query, convert_to_tensor=True)

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    # Print time taken
    if print_time:
        print(f'[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds')

    # Get top k resources
    scores, indices = torch.topk(input=dot_scores, k=n_resources)

    return scores, indices


def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: List[Dict],
                                 scores: torch.tensor,
                                 indices: torch.tensor,
                                 n_resources: int = 5):
    """
    Prints top N results and their scores based on the query.
    """
    for score, idx in zip(scores, indices):
        print(f'[INFO] Score: {score:.4f}')
        print('Text:')
        print(pages_and_chunks[idx]['sentence_chunk'])
        print(f'Page number: {pages_and_chunks[idx]["page_number"]}')

In [44]:
query = 'handwash'
retrieve_relevant_resources(query=query, embeddings=embeddings)
print_top_results_and_scores(query=query, embeddings=embeddings)

[INFO] Time taken to get scores on 237 embeddings: 0.00027 seconds


TypeError: print_top_results_and_scores() missing 3 required positional arguments: 'pages_and_chunks', 'scores', and 'indices'

In [46]:
from timeit import default_timer as timer
import torch
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict

# Assume embeddings and pages_and_chunks are defined elsewhere in your code
# embeddings = ... (a tensor with embedded sentences)
# pages_and_chunks = ... (a list of dictionaries with 'sentence_chunk' and 'page_number' for each)

def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer = None,
                                n_resources: int = 5,
                                print_time: bool = True):
    """
    Embeds a query with the embed model and retrieves relevant resources.
    """
    # Embed query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    # Print time taken
    if print_time:
        print(f'[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds')

    # Get top k resources
    scores, indices = torch.topk(input=dot_scores, k=n_resources)

    return scores, indices


def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: List[Dict],
                                 scores: torch.tensor,
                                 indices: torch.tensor,
                                 n_resources: int = 5):
    """
    Prints top N results and their scores based on the query.
    """
    for score, idx in zip(scores, indices):
        print(f'[INFO] Score: {score:.4f}')
        print('Text:')
        print(pages_and_chunks[idx]['sentence_chunk'])
        print(f'Page number: {pages_and_chunks[idx]["page_number"]}')

In [47]:
query = 'handwash'
# Assuming `embeddings` and `pages_and_chunks` are already defined
scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings, model=embed_model)
print_top_results_and_scores(query=query, embeddings=embeddings, pages_and_chunks=pages_and_chunks, scores=scores, indices=indices)

[INFO] Time taken to get scores on 237 embeddings: 0.00018 seconds
[INFO] Score: 0.6054
Text:
Beverage stations are excluded. 17.5.4 Equipment and Supplies The handwashing station must include a handwash sink, soap dispenser, and single-use paper towel dispenser. Electric hand dryers can be installed in addition to paper towel dispensers. Waste receptacles must be provided in close proximity to the handwash sink and sized to accommodate the quantity of paper towel waste generated. The handwashing station may be decorative but must be nonabsorbent, durable, and EASILY CLEANABLE. 17.5.5 Automatic Handwashing System An automatic handwashing system in lieu of a handwash sink is acceptable. 17.5.6 Sign Each handwashing station must have a sign advising passengers to wash hands before eating. A pictogram can be used in lieu of words on the sign.
Page number: 67
[INFO] Score: 0.6021
Text:
21.6 General Hygiene Construct handwashing stations in the following areas according to section 7.1. 21.6

## Getting an LLM for local generation